# JS05 - Graph Traversals

## Overview

A graph consists of vertices and edges. Vertices are stored as documents in vertex collections and edges stored as documents in edge collections. The collections used in a graph and their relations are specified with edge definitions.

In this tutorial we will build a simple dating application caled "Hopefulls", and then use graph traversals to see who has liked whom.

## Pre-requisite

Lets Assume 
- you have already made a tenant account, and have a username and password
- you have installed the jsc8 drivers as explained in section 01
- you have generated an API Key as explained in section 01


In [ ]:
#/* run this once to install javascript kernal and jsc8 in google colab, then reload, and then skip this
!npm install jsc8
!npm install -g --unsafe-perm ijavascript
!ijsinstall --install=global  # */

## Step 05-A Importing Libraries & Define Variables

In [1]:
// Variables - Queries
const global_url = "gdn.paas.macrometa.io";
const email = "email"; // <-- Email goes here
const password = "password"; // <-- password goes here
const geo_fabric = "_system";
const collection_singles = "Singles";
const collection_likes = "Likes";
const collection_graph = "Hopefulls_graph";
const read_singles = "FOR singles IN Singles RETURN singles";
const read_likes = "FOR likes IN Likes RETURN like";

const Hopefulls_Client = "SanJoseLou";
const Hopefulls_Client2 = "StocktonPeria";

const graph_traversal1 = `FOR c IN Singles FILTER c.hopefulls_user_name == \"${Hopefulls_Client}\" FOR v IN 1..1 INBOUND c Likes RETURN v`;
const graph_traversal2 = `FOR c IN Singles FILTER c.hopefulls_user_name == \"${Hopefulls_Client}\" FOR v IN 1..1 OUTBOUND c Likes RETURN v`;


## Step 05-B - Connecting to GDN

In [2]:
const jsc8 = require("jsc8");

// Simple Way
const client = new jsc8({
  url: "https://verizon.5g.macrometa.io",
  token: "XXXX",
  fabricName: "_system",
});

// ----- OR -----
// const client = new jsc8({url: "https://verizon.5g.macrometa.io", apiKey: "XXXX", fabricName: '_system'});

// To use advanced options
// const client = new jsc8("https://verizon.5g.macrometa.io");
// client.login(email, password).then(result => console.log("Login successfully", result)).catch(err => console.error("Error while login", err.message))


## Step 05-C - Creating Collections

In [3]:
async function createCollection() {
  console.log("\n 2. CREATE_COLLECTION");

  try {
    console.log(
      `Creating the CREATE_SINGLES_VERTEX_COLLECTION ${collection_singles}...`
    );
    let exists_coll = await client.hasCollection(collection_singles);
    if (!exists_coll) {
      await client.createCollection(collection_singles);
    }

    console.log(
      `Creating the CREATE_LIKES_EDGE_COLLECTION ${collection_likes}...`
    );
    exists_coll = await client.hasCollection(collection_likes);
    if (!exists_coll) {
      await client.createCollection(collection_likes, {}, true);
    }
  } catch (e) {
    console.log("Collection creation did not succeed due to " + e.message);
  }
}

createCollection();


## Step 05-D - Inserting Data - "Singles"

In [4]:
let people_data = [
  {
    full_name: "Kiel Dummer",
    first_name: "Kiel",
    last_name: "Dummer",
    city: "Burbank",
    state: "CA",
    address: "40317 5th Crossing",
    hopefulls_user_name: "BurbankKeil",
    _key: "BurbankKeil",
  },
  {
    full_name: "Pernell Winspare",
    first_name: "Pernell",
    last_name: "Winspare",
    city: "San Diego",
    state: "CA",
    address: "596 Packers Pass",
    hopefulls_user_name: "SanDiegoPernell",
    _key: "SanDiegoPernell",
  },
  {
    full_name: "Ava Kermath",
    first_name: "Ava",
    last_name: "Kermath",
    city: "Berkeley",
    state: "CA",
    address: "2 Doe Crossing Junction",
    hopefulls_user_name: "BerkeleyAva",
    _key: "BerkeleyAva",
  },
  {
    full_name: "Tremain McGrah",
    first_name: "Tremain",
    last_name: "McGrah",
    city: "Torrance",
    state: "CA",
    address: "079 Russell Street",
    hopefulls_user_name: "TorranceTremain",
    _key: "TorranceTremain",
  },
  {
    full_name: "Vidovik Boddam",
    first_name: "Vidovik",
    last_name: "Boddam",
    city: "Los Angeles",
    state: "CA",
    address: "3 Brentwood Crossing",
    hopefulls_user_name: "LosAngelesVidovik",
    _key: "LosAngelesVidovik",
  },
  {
    full_name: "Oralie Goward",
    first_name: "Oralie",
    last_name: "Goward",
    city: "Los Angeles",
    state: "CA",
    address: "922 Columbus Park",
    hopefulls_user_name: "LosAngelesOralie",
    _key: "LosAngelesOralie",
  },
  {
    full_name: "Lou Feaveer",
    first_name: "Lou",
    last_name: "Feaveer",
    city: "San Jose",
    state: "CA",
    address: "55223 Hooker Crossing",
    hopefulls_user_name: "SanJoseLou",
    _key: "SanJoseLou",
  },
  {
    full_name: "Peria King",
    first_name: "Peria",
    last_name: "King",
    city: "Stockton",
    state: "CA",
    address: "8 Troy Plaza",
    hopefulls_user_name: "StocktonPeria",
    _key: "StocktonPeria",
  },
];

async function insertDocumentMany() {
  console.log(`\n 3a. INSERT_SINGLES_DATA in region ${global_url}`);
  await client.insertDocumentMany(collection_singles, people_data);
}

insertDocumentMany();


## Step 05-E - Inserting Data - "Who Likes Whom"

In [5]:
let callsData = [
  {
    hopefulls_user_name: "BurbankKeil",
    singles_liked: "SanJoseLou",
    _from: "Singles/BurbankKeil",
    _to: "Singles/SanJoseLou",
    liked_date: "1/4/2020",
    liked_time: "13:33",
    liked_amount: 30,
  },
  {
    hopefulls_user_name: "SanJoseLou",
    singles_liked: "StocktonPeria",
    _from: "Singles/SanJoseLou",
    _to: "Singles/StocktonPeria",
    liked_date: "1/28/2020",
    liked_time: "3:02",
    liked_amount: 18,
  },
  {
    hopefulls_user_name: "BerkeleyAva",
    singles_liked: "StocktonPeria",
    _from: "Singles/BerkeleyAva",
    _to: "Singles/StocktonPeria",
    liked_date: "1/28/2020",
    liked_time: "3:02",
    liked_amount: 18,
  },
];

async function insertDocumentMany() {
  console.log(`\n 3a. INSERT_LIKES_DATA in region ${global_url}`);
  await client.insertDocumentMany(collection_likes, callsData);
}

insertDocumentMany();


## Step 05-F - Creating a Graph

In [6]:
//Step4: Create a graph

async function createGraph() {
  console.log(`\n 4. CREATE_GRAPH region ${global_url}`);

  await client.createGraph(collection_graph, {
    edgeDefinitions: [
      {
        collection: collection_likes,
        from: [collection_singles],
        to: [collection_singles],
      },
    ],
  });
  console.log("Graph created");
}

createGraph().catch((err) =>
  console.log("Error while creating graph :", err.message)
);


## Step 05-G - Execute a Traversal - "Find Singles who liked Lou?"

In [7]:
async function readData() {
  console.log(
    `5a. GRAPH_TRAVERSAL: Find Singles who liked: ${Hopefulls_Client2}`
  );
  let result = await client.executeQuery(graph_traversal1);
  console.log(result);
}

readData().catch((err) =>
  console.log("Error while execute a traversal :", err.message)
);


## Step 05-H - Execute a Traversal - "Find Singles who Lou Liked?"

In [8]:
async function readData() {
  console.log(
    `5b. GRAPH_TRAVERSAL: Find Singles whole were liked from: ${Hopefulls_Client}`
  );
  result = await client.executeQuery(graph_traversal2);
  console.log(result);
}

readData().catch((err) =>
  console.log("Error while execute a traversal :", err.message)
);


## Step 05-I - Delete Data and Collections

In [9]:
// Step6: Delete Data

async function deleteData() {
  console.log(" 6. DELETE_DATA");
  await client.deleteGraph(collection_graph, true);
  console.log("Collection and Graph deleted");
}

deleteData().catch((err) =>
  console.log("Error while deleting collection and graphs:", err.message)
);


## Section Completed!

TBC...